In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import spikeinterface.sorters
import spikeinterface.full as si
import  scipy.signal
import spikeinterface.extractors as se
import spikeinterface.comparison
import spikeinterface.exporters
import spikeinterface.curation
import spikeinterface.widgets 
import docker
from datetime import datetime
import itertools
import scipy.io as sio
import sys
import glob
import ast
import pandas as pd
import matplotlib.pyplot as plt
from wavpack_numcodecs import WavPack
compressor_wv = WavPack(level=3, bps=None)
# The first command-line argument after the script name is the mouse identifier.
mouse='M00013'
# All command-line arguments after `mouse` and before `save_date` are considered dates.

save_date_list=[20250218]

base_folder='/run/user/1004/gvfs/smb-share:server=rdp.arc.ucl.ac.uk,share=ritd-ag-e=e=ritd-ag-project-ee=ritd-agee=ritd-ag-project-rd01ie-asale69/ibn-vision/DATA/SUBJECTS/' 
base_folder='/run/user/1004/gvfs/smb-share:server=rdp.arc.ucl.ac.uk,share=ritd-ag-project-rd01qp-dbend52/Ellie/DATA/SUBJECTS/' 
no_probe = 2
# slice off the reocrding that had failing errors happened during the recording
no_dates = len(save_date_list)

date_count = 0
for save_date_tmp in save_date_list:
    save_date = str(save_date_tmp)
    recording_folder = base_folder + mouse + '/ephys/' + save_date + '/'
    error_info_path = os.path.join(recording_folder, 'error_info.csv')
    if os.path.exists(error_info_path):
        error_info = pd.read_csv(error_info_path)
        print(error_info)
    else:
        print(f'No error_info.csv found in {recording_folder}')
        columns = ['acquisition', 'session', 'error_sample', 'error_timestamp']
        error_info = pd.DataFrame(columns=columns)
        error_info.to_csv(error_info_path, index=False)
        print(f'Created empty error_info.csv in {recording_folder}')
    num_rows = len(error_info)
    if num_rows > 0:
        print('error exists in recordings. Will slice recordings after error happened')
        unique_acquisition_indices = error_info.drop_duplicates(subset='acquisition', keep='first').index.tolist()
        print(f'Unique acquisition indices: {unique_acquisition_indices}')
        acquisition_to_slice = error_info['acquisition'].tolist()
        segment_to_slice = error_info['session'].apply(lambda x: int(x[-1])).tolist()
        error_sample = error_info['error_sample'].tolist()
        acquisition_to_slice = [acquisition_to_slice[i] for i in unique_acquisition_indices]
        segment_to_slice = [segment_to_slice[i] for i in unique_acquisition_indices]
        error_sample = [error_sample[i] for i in unique_acquisition_indices]
    else:
        acquisition_to_slice = []
        segment_to_slice = []
        error_sample = []
    acquisition_folders = []
 
    acquisition_base_path = base_folder + mouse + '/ephys/' + save_date + '/*' + save_date
    print(acquisition_base_path)
    acquisition_folders = glob.glob(acquisition_base_path + '_*')
    print('acquisition_folders: ', acquisition_folders)
 
    for probe in range(int(no_probe)):
        for acquisition_folder in acquisition_folders:
                # Rename all tcat files to t0 if they exist
            tcat_pattern = os.path.join(acquisition_folder,'**','*tcat.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            # Step 1: Iterate over the list of files with tcat in the name
            for old_name in files_to_rename:
                # Step 2: Construct the new filename (REMEMBER to switch the name back to tcat)
                new_name = old_name.replace('tcat', 't0')
                # Step 3: Rename the file
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            recording_raw = si.read_spikeglx(acquisition_folder,stream_name='imec' + str(probe) + '.ap')
            no_segments = recording_raw.get_num_segments()
            segments = [i for i in range(no_segments)]
            num_segments = [recording_raw.get_num_frames(segment_index=i) for i in segments]
            end_sample = list(itertools.accumulate(num_segments))
 
            print('reading recording from:', acquisition_folder,'and probe',str(probe))
            if any(f'{acq}' in acquisition_folder[-1] for acq in acquisition_to_slice):
                print(f'Acquisition {acquisition_folder} corresponds to one of the acquisition_to_slice: {acquisition_to_slice}')
 
                slice_index = acquisition_to_slice.index(int(acquisition_folder[-1]))
                sample_index = error_sample[slice_index]
                segment_index = segment_to_slice[slice_index]
                print(f'Removing samples for acquisition {acquisition_folder[-1]} segment {segment_index} after {sample_index-300000} th sample')
                print(f'recording before slicing: {recording_raw}')
                selected_segment =  si.select_segment_recording(recording_raw,segment_indices=segment_index)
                sliced_segment =selected_segment.frame_slice( start_frame=0, end_frame=sample_index-300000 )
                print(f'appending sliced segment from segment {segment_index} to previous recording segment {list(range(segment_index))}')
                if segment_index > 0:
                    previous_recording = si.select_segment_recording(recording_raw,segment_indices = list(range(segment_index)))
                    recording_raw = si.append_recordings([previous_recording,sliced_segment])
                else:
                    recording_raw = sliced_segment
            print(f'sliced recording from acquisition {acquisition_folder}:')
            print(recording_raw)
            print(recording_raw.get_num_segments)
            compression_folder = base_folder + mouse + '/ephys/' + save_date + '/probe' + str(probe) + '_compressed' + acquisition_folder[-2:]
            print('compressing to folder: '+compression_folder)
            if not os.path.exists(compression_folder + '.zarr'):
                print(f'Compression folder {compression_folder} does not exist. Saving compressed recording.')
                raw_compressed = recording_raw.save(format="zarr", folder=compression_folder, compressor=compressor_wv, n_jobs=16, chunk_duration="1s")
            else:
                print(f'Compression folder {compression_folder} already exists. Skipping save.')
            tcat_pattern = os.path.join(acquisition_folder, '**', '*t0.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            for old_name in files_to_rename:
                new_name = old_name.replace('t0', 'tcat')    
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            load_compressed = si.read_zarr(compression_folder+'.zarr')
            recording_raw.get_probe().to_dataframe()
            fig, ax = plt.subplots(figsize=(15, 10))
            si.plot_probe_map(recording_raw, ax=ax)
            ax.set_ylim(-100, 4000)
    date_count = date_count + 1

ModuleNotFoundError: No module named 'spikeinterface'

In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import spikeinterface.sorters
import spikeinterface.full as si
import  scipy.signal
import spikeinterface.extractors as se
import spikeinterface.comparison
import spikeinterface.exporters
import spikeinterface.curation
import spikeinterface.widgets 
import docker
from datetime import datetime
import itertools
import scipy.io as sio
import sys
import glob
import ast
import pandas as pd
import matplotlib.pyplot as plt
from wavpack_numcodecs import WavPack
compressor_wv = WavPack(level=3, bps=None)
# The first command-line argument after the script name is the mouse identifier.
mouse='M24016'
# All command-line arguments after `mouse` and before `save_date` are considered dates.

save_date_list=[20240625,20240621,20240622,20240624,20240627,20240628,20240701,20240702,20240703]

base_folder='/saleem/ibn-vision/DATA/SUBJECTS/' 
no_probe = 2
# slice off the reocrding that had failing errors happened during the recording
#acquisition_to_slice = []
#slice_time_ranges = []
no_dates = len(save_date_list)


date_count = 0
for save_date_tmp in save_date_list:
    acquisition_to_slice = []

    save_date = str(save_date_tmp)
    recording_folder = base_folder + mouse + '/ephys/' + save_date + '/'
    error_info_path = os.path.join(recording_folder, 'error_info.csv')
    if os.path.exists(error_info_path):
        error_info = pd.read_csv(error_info_path)
        print(error_info)
    else:
        print(f'No error_info.csv found in {recording_folder}')
        columns = ['acquisition', 'session', 'error_sample', 'error_timestamp']
        error_info = pd.DataFrame(columns=columns)
        error_info.to_csv(error_info_path, index=False)
        print(f'Created empty error_info.csv in {recording_folder}')
    num_rows = len(error_info)
    if num_rows > 0:
        print('error exists in recordings. Will slice recordings after error happened')
        unique_acquisition_indices = error_info.drop_duplicates(subset='acquisition', keep='first').index.tolist()
        print(f'Unique acquisition indices: {unique_acquisition_indices}')
        acquisition_to_slice = error_info['acquisition'].tolist()
        segment_to_slice = error_info['session'].apply(lambda x: int(x[-1])).tolist()
        error_sample = error_info['error_sample'].tolist()
        acquisition_to_slice = [acquisition_to_slice[i] for i in unique_acquisition_indices]
        segment_to_slice = [segment_to_slice[i] for i in unique_acquisition_indices]
        error_sample = [error_sample[i] for i in unique_acquisition_indices]
    acquisition_folders = []

    acquisition_base_path = base_folder + mouse + '/ephys/' + save_date + '/*' + save_date
    print(acquisition_base_path)
    acquisition_folders = glob.glob(acquisition_base_path + '_*')
    print('acquisition_folders: ', acquisition_folders)

    for probe in range(int(no_probe)):
        for acquisition_folder in acquisition_folders:

            tcat_pattern = os.path.join(acquisition_folder, '**', '*t0.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            for old_name in files_to_rename:
                new_name = old_name.replace('t0', 'tcat')    
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')

write_zarr_recording 
n_jobs=16 - samples_per_chunk=29,999 - chunk_memory=21.97 MiB - total_memory=351.55 MiB - chunk_duration=1.00s (999.97 ms)


write_zarr_recording: 100%|████████████████████████████████████████| 12376/12376 [54:20<00:00,  3.80it/s]


write_zarr_recording 
n_jobs=16 - samples_per_chunk=29,999 - chunk_memory=21.97 MiB - total_memory=351.55 MiB - chunk_duration=1.00s (999.97 ms)


write_zarr_recording: 100%|████████████████████████████████████████| 12376/12376 [59:15<00:00,  3.48it/s]
